# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откроем файл с данными и изучим общую информацию

In [1]:
import pandas as pd

server_path = '/datasets/data.csv'
filename = 'data.csv'
try:
    data = pd.read_csv(server_path)
except: 
    data = pd.read_csv(filename)
    
display(data.info())
display(data.head())
data.to_csv('data.csv')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
Unnamed: 0          21525 non-null int64
Unnamed: 0.1        21525 non-null int64
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(7), object(5)
memory usage: 2.3+ MB


None

,Unnamed: 0,Unnamed: 0.1,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,0,0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,2,2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,4,4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

В таблице 12 переменных и 21525 наблюдений, в переменных стажа работы и ежемесячного дохода обнаружены пропуски.
Также в превью таблицы найдены необычные значения переменной стажа работы: встречаются как отрицательные значения, так и во много раз превышающие возраст заемщика. Это может быть связано с некорректным преобразованием дат при расчете timedelta между трудоустройством и увольнением или другими ошибками первичной обработки данных. Следует выяснить причину и устранить ошибку в исходных данных.
Значения переменной образования по-разному закодированы (в верхнем и нижнем регистре).

## Шаг 2. Предобработка данных

### Обработка пропусков

In [2]:
# получим информацию о пропусках в данных
print('Количество пропусков:')
print(data.isnull().sum())
print('Пересечения в пропусках стажа и дохода:')
print(data[(data['days_employed'].isnull()) & (data['total_income'].isnull())].count())

# заменим пропуски в доходе на медианные значения по группам income_type
income_type_group = data.groupby('income_type').agg({'total_income' : 'median'})
for inc_type in income_type_group.index:
    data.loc[data['income_type'] == inc_type,'total_income'] = data.loc[data['income_type'] == inc_type,'total_income'].fillna(income_type_group.loc[inc_type, 'total_income'])    

# заменим пропуски в стаже на медианные значения
days_employed_median = data['days_employed'].median()
data['days_employed'] = data['days_employed'].fillna(value = days_employed_median)

print('Количество пропусков после замены:')
print(data.isnull().sum())

# Проверяем аномальные значения в числовых переменных
print('Значения переменной chidren: {}'.format(data['children'].unique()))
print('Значения переменной debt: {}'.format(data['debt'].unique()))
print('Значения переменной dob_years: {}'.format(data['dob_years'].unique()))
print('Минимальное значение переменной total_income: {}'.format(data['total_income'].min()))
print('Максимальное значение переменной total_income: {}'.format(data['total_income'].max()))

# Распечатать строки dob_years 0
print('Количество записей возраста: {}'.format(data[data['dob_years']== 0]['dob_years'].value_counts()))
display(data[data['dob_years']== 0].head(10))

# Заменить dob_years==0 на медианный возраст по income type)
income_type_age = data.groupby('income_type').agg({'dob_years' : 'median'})
for inc_type in income_type_age.index:
    data.loc[data['income_type'] == inc_type,'dob_years'] = data.loc[data['income_type'] == inc_type,'dob_years'].replace(0, income_type_age.loc[inc_type, 'dob_years'])    

# Исправить children -1
print('Количество записей children: {}'.format(data[data['children']== -1]['children'].value_counts()))
display(data[data['children']== -1].head(10))
data['children'] = data['children'].abs()

Количество пропусков:
Unnamed: 0             0
Unnamed: 0.1           0
children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64
Пересечения в пропусках стажа и дохода:
Unnamed: 0          2174
Unnamed: 0.1        2174
children            2174
days_employed          0
dob_years           2174
education           2174
education_id        2174
family_status       2174
family_status_id    2174
gender              2174
income_type         2174
debt                2174
total_income           0
purpose             2174
dtype: int64
Количество пропусков после замены:
Unnamed: 0          0
Unnamed: 0.1        0
children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
f

,Unnamed: 0,Unnamed: 0.1,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,99,99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,149,149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,270,270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,578,578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,1040,1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,1149,1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,1175,1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,1386,1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,1890,1890,0,-1203.369529,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,142594.396847,жилье
1898,1898,1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


Количество записей children: -1    47
Name: children, dtype: int64


,Unnamed: 0,Unnamed: 0.1,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,291,291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,705,705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,742,742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,800,800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,941,941,-1,-1203.369529,57,Среднее,1,женат / замужем,0,F,пенсионер,0,118514.486412,на покупку своего автомобиля
1363,1363,1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,1929,1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,2073,2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,3814,3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,4201,4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


**Вывод**

Все пропуски в доходах и опыте работы присутствуют в одних и тех же наблюдениях. Это может быть связано с непредоставленной заемщиком справкой от работодателя или неофициальным трудоустройством.
Если удалить все наблюдения с пропусками, это приведет к потере 10% данных. Поэтому пропущенные значения были заменены.
В переменной ежемесячного дохода пропуски заменены на медианные значения в разрезе типа занятости, поскольку можно ожидать, что внутри групп доход более однородный. 
Поскольку переменная стажа не используется в дальнейшем анализе и заполнена значениями, не поддающимися интерпретации (отрицательные, а также в десятки раз превышающие возраст значения), можно прибегнуть к формальной замене, чтобы пропуски не мешали дальнейшему анализу по другим параметрам. Поэтому пропуски были заменены на медиану всех наблюдений.
Также в данных обнаружены неявные пропуски: в переменной возраста найдено 101 нулевое значения, что фактически означает остутствие значений. Нулевые значения были заменены на медиану по категориям занятости, так как внутри этих групп возраст более однородный.
В переменной количества детей найдено 47 записей со значением -1. Поскольку отрацательные значения в этой переменной невозможны, они были преобразованы в модуль числа.

### Замена типа данных

In [3]:
data['total_income'] = data['total_income'].astype('int')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 14 columns):
Unnamed: 0          21525 non-null int64
Unnamed: 0.1        21525 non-null int64
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 2.3+ MB


**Вывод**

Исходные данные представлены 5 категориальными, 2 вещественными и 5 целочисленных переменных. Чтобы увеличить точность, для переменной дохода тип данных заменен с вещественного на целочисленный.

### Обработка дубликатов

In [4]:
# поиск неявных дубликатов и мусора в категориях
print(data['education'].unique())
print(data['family_status'].unique())
print(data['gender'].unique())
print(data['income_type'].unique())
print(data['purpose'].unique())

# унификация категорий образования
data['education'] = data['education'].str.lower()
print(data['education'].unique())

# замена некрасивых значений в gender и family status
data['gender'] = data['gender'].replace('XNA', 'None')
data['family_status'] = data['family_status'].str.lower()

# Проверим соответствие education и education_id  
print(data.groupby('education')['education_id'].value_counts())

# Проверим соответствие family_status и family_status_id 
print(data.groupby('family_status')['family_status_id'].value_counts())

print('Дубликатов в таблице: {}'.format(data.duplicated().sum()))
data = data.drop_duplicates().reset_index(drop=True)

['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']
['женат / замужем' 'гражданский брак' 'вдовец / вдова' 'в разводе'
 'Не женат / не замужем']
['F' 'M' 'XNA']
['сотрудник' 'пенсионер' 'компаньон' 'госслужащий' 'безработный'
 'предприниматель' 'студент' 'в декрете']
['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образо

**Вывод**

Для удаления неявных дубликатов и мусора в категориях произведена унификация категорий образования (переводом в нижний регистр). 
Переменные education_id и family_status_id закодированы корректно (для каждого типа education и family_status используются единые коды).
После унификации категориальных переменных обнаружено и удалено 71 полных дубликата, которые могли появиться вследствие сбоя в записи данных.

### Лемматизация

In [5]:
from pymystem3 import Mystem
m = Mystem()
data['lemmas'] = data['purpose'].apply(m.lemmatize)

purpose_dict = data.loc[:, ['lemmas','purpose']]
purpose_dict['lemmas']= purpose_dict['lemmas'].apply(' '.join)
purpose_dict = purpose_dict.drop_duplicates().reset_index(drop=True)

all_lemmas = ' '.join(purpose_dict['lemmas']).split()
      
from collections import Counter
print(Counter(all_lemmas)) # это самые частые слова

Counter({'покупка': 10, 'недвижимость': 10, 'автомобиль': 9, 'образование': 9, 'жилье': 7, 'с': 5, 'операция': 4, 'на': 4, 'свой': 4, 'свадьба': 3, 'строительство': 3, 'получение': 3, 'высокий': 3, 'дополнительный': 2, 'для': 2, 'коммерческий': 2, 'жилой': 2, 'заниматься': 2, 'сделка': 2, 'приобретение': 1, 'сыграть': 1, 'проведение': 1, 'семья': 1, 'собственный': 1, 'подержать': 1, 'со': 1, 'подержанный': 1, 'профильный': 1, 'сдача': 1, 'ремонт': 1})


**Вывод**

В исходных данных для обозначения аналогичных целей получения кредита используются разные формулировки. Для удобства анализа заемщиков необходимо создать унифицированный справочник целей. Для этого сгенерированны леммы каждого значения purpose, после чего создан список из 37 уникальных лемм и подсчтана частотность всех слов в них. В результате можно составить список ключевых слов, маркирующих разные цели кредитования (таких как автомобиль, недвижимость/жилье, образование, ремонт, свадьба).

### Категоризация данных

In [6]:
def purpose_group(purpose_lemma):
    if 'недвижимость' in purpose_lemma or 'жилье'in purpose_lemma:
        if 'ремонт' in purpose_lemma:
            return 'ремонт'
        return 'покупка недвижимости'
    if 'образование' in purpose_lemma:
        return 'образование'
    if 'автомобиль'in purpose_lemma:
        return 'покупка автомобиля'
    if 'свадьба'in purpose_lemma:
        return 'свадьба'
    return '000'

purpose_dict['purpose_group'] = purpose_dict['lemmas'].apply(purpose_group)

data = data.merge(purpose_dict, on='purpose', how='left')

def children_group(children):
    if children == 0:
        return 'нет детей'
    return 'есть дети'

data['children_group'] = data['children'].apply(children_group)

display(data['total_income'].quantile([0.20, 0.40, 0.60, 0.80])) # примерные границы 100000, 130000, 160000, 210000

def income_group(income):
    if income <= 100000:
        return 'до 100000'
    if (income > 100000) & (income <= 130000):
        return 'от 100000 до 130000'
    if (income > 130000) & (income <= 160000):
        return 'от 130000 до 160000'
    if (income > 160000) & (income <= 210000):
        return 'от 160000 до 210000'
    if income > 210000:
        return 'более 210000'

data['income_group'] = data['total_income'].apply(income_group)

0.2     98661.6
0.4    132141.8
0.6    161151.2
0.8    214269.6
Name: total_income, dtype: float64

**Вывод**

Для удобства дальнейшего анализа были созданы категоризированные переменные целей получения кредита, уровня дохода и наличия детей.

По результатам лемматизации цели получения кредита можно укрупненно отнести к 5 категориям: покупка недвижимости, ремонт, образование, покупка автомобиля, свадьба. Слово "недвижимость" встречается в леммах, связанных и с покупкой, и с ремонтом. Чтобы разделить категории, использована дополнительное условие со словом "ремонт".

Чтобы сбаланированно разбить наблюдения на категории по уровню доходов, были посчтианы границы 5 квантилей переменной total_income. Округлив их значения до десятков тысяч рублей, получем границы категорий: 100000, 130000, 160000, 210000.

## Шаг 3. Анализ данных

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [7]:
children_group_pivot = data.pivot_table(index=['children_group'], columns='debt', values='purpose', aggfunc='count', fill_value=0, margins=True)
children_group_pivot.set_axis(['Нет долга', 'Есть долг', 'Все заемщики'], axis = 'columns', inplace = True)
children_group_pivot['Процент должников'] = (children_group_pivot['Есть долг']/children_group_pivot['Все заемщики']).apply(lambda x: '{:.2%}'.format(x))
display(children_group_pivot)

children_pivot = data.pivot_table(index=['children'], columns='debt', values='purpose', aggfunc='count', fill_value=0, margins=True)
children_pivot.set_axis(['Нет долга', 'Есть долг', 'Все заемщики'], axis = 'columns', inplace = True)
children_pivot['Процент должников'] = (children_pivot['Есть долг']/children_pivot['Все заемщики']).apply(lambda x: '{:.2%}'.format(x))
display(children_pivot)

,Нет долга,Есть долг,Все заемщики,Процент должников
children_group,,,,
есть дети,6698,678,7376,9.19%
нет детей,13086,1063,14149,7.51%
All,19784,1741,21525,8.09%


,Нет долга,Есть долг,Все заемщики,Процент должников
children,,,,
0,13086,1063,14149,7.51%
1,4420,445,4865,9.15%
2,1861,194,2055,9.44%
3,303,27,330,8.18%
4,37,4,41,9.76%
5,9,0,9,0.00%
20,68,8,76,10.53%
All,19784,1741,21525,8.09%


**Вывод**

Среди заемщиков без детей доля должников меньше, чем среди заемщиков с детьми. При этом заемщики с 2 детьми в среднем чаще становятся должниками, чем заемщики с 1 ребенком. Поскольку заемщики с 3 и более детьми представлены небольшим числом наблюдений, выводы о более высокой конверсии в должников могут быть некорректными.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [8]:
family_pivot = data.pivot_table(index=['family_status'], columns='debt', values='purpose', aggfunc='count', fill_value=0)
family_pivot.set_axis(['Нет долга', 'Есть долг'], axis = 'columns', inplace = True)
family_pivot['Все заемщики'] = family_pivot['Есть долг']+ family_pivot['Нет долга']
family_pivot['Процент должников'] = (family_pivot['Есть долг']/family_pivot['Все заемщики']).apply(lambda x: '{:.2%}'.format(x))
display(family_pivot.sort_values(by='Процент должников', ascending = False))

,Нет долга,Есть долг,Все заемщики,Процент должников
family_status,,,,
не женат / не замужем,2539,274,2813,9.74%
гражданский брак,3789,388,4177,9.29%
женат / замужем,11449,931,12380,7.52%
в разводе,1110,85,1195,7.11%
вдовец / вдова,897,63,960,6.56%


**Вывод**

Женатые/замужние заемщики в среднем реже становятся должниками, чем не состоящие в браке (не женатые/не замужние или в гражданском браке) и чаще, чем разведенные или вдовцы.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [9]:
income_pivot_mean = data.pivot_table(columns='debt', values='total_income', aggfunc='mean')
income_pivot_mean.set_axis(['Нет долга', 'Есть долг'], axis = 'columns', inplace = True)
print(income_pivot_mean)

income_pivot = data.pivot_table(index=['income_group'], columns='debt', values='purpose', aggfunc='count', fill_value=0).reset_index()
income_pivot.set_axis(['income_group','Нет долга', 'Есть долг'], axis = 'columns', inplace = True)
income_pivot['Все заемщики'] = income_pivot['Есть долг']+ income_pivot['Нет долга']
income_pivot['Процент должников'] = (income_pivot['Есть долг']/income_pivot['Все заемщики']).apply(lambda x: '{:.2%}'.format(x))

sort_dict = {'до 100000':0, 'от 100000 до 130000':1, 'от 130000 до 160000':2, 'от 160000 до 210000':3, 'более 210000':4}
income_pivot.iloc[income_pivot['income_group'].map(sort_dict).sort_values().index]


                  Нет долга      Есть долг
total_income  165582.662859  161158.393452


,income_group,Нет долга,Есть долг,Все заемщики,Процент должников
1,до 100000,4109,354,4463,7.93%
2,от 100000 до 130000,3565,328,3893,8.43%
3,от 130000 до 160000,4051,391,4442,8.80%
4,от 160000 до 210000,3871,353,4224,8.36%
0,более 210000,4188,315,4503,7.00%


**Вывод**

В среднем доход заемщиков без долга незначительно превышает доход заемщиков с задолженностью. Однако если анализировать своевременность возврата кредитов в разрезе категорий по уровню доходов, отличия более заметны. Так, наименьшая доля должников среди заемщиков с доходом более 210000 рублей, а наибольшая - у заемщиков с доходом от 130000 до 160000 рублей. Менее очевидное наблюдение - относительно низкая доля должников в категории с доходами до 100000 рублей. Следовательно, явной зависимости своевременности возврата кредита от уровня доходов нет.

- Как разные цели кредита влияют на его возврат в срок?

In [10]:
purpose_pivot = data.pivot_table(index=['purpose_group'], columns='debt', values='purpose', aggfunc='count', fill_value=0)
purpose_pivot.set_axis(['Нет долга', 'Есть долг'], axis = 'columns', inplace = True)
purpose_pivot['Все заемщики'] = purpose_pivot['Есть долг']+ purpose_pivot['Нет долга']
purpose_pivot['Процент должников'] = (purpose_pivot['Есть долг']/purpose_pivot['Все заемщики']).apply(lambda x: '{:.2%}'.format(x))
display(purpose_pivot.sort_values(by='Процент должников', ascending = False))

,Нет долга,Есть долг,Все заемщики,Процент должников
purpose_group,,,,
покупка автомобиля,3912,403,4315,9.34%
образование,3652,370,4022,9.20%
свадьба,2162,186,2348,7.92%
покупка недвижимости,9481,747,10228,7.30%
ремонт,577,35,612,5.72%


**Вывод**

Кредиты, выданные на покупку автомобиля, реже всего возвращаются в срок. Вторая по доле должников категория - кредиты на образовательные цели. Свадьба находится на третьем месте по доле должников. Покупка недвижимости - на четвертом. Кредиты на ремонт лидируют по своевременному возврату.

## Шаг 4. Общий вывод

На этапе предобработки в данных были обнаружены и исправлены пропуски, дубликаты, а также некорректно закодированные категорильные переменные и аномальные значения (отрицательное количество дней трудового стажа, возраст 0 лет, -1 ребенок). С целью сохранения данных все пропущенные значения были заменены характерными значениями для предствителей соотвествующих групп занятых. Также для удобства анализа были перекодированы/созданы новые справочники образования, уровня доходов, наличия детей и целей кредитования.

Согласно полученным результатам анализа, наиболее рисковые категории заемщиков - не состоящие в браке клиенты с детьми и доходом от 130000 до 160000, получающие кредит на покупку автомобиля или образование. Наиболее надежными заемщиками являются - женатые/замужние клиенты без детей, а также разведенные или вдовцы с доходом более 210000, обращающиесяза кредитом на покупку или ремонт недвижимости.
Посокольку явной зависимости своевременности возврата кредита от уровня доходов не обнаружено, при построении модели кредитного скоринга следует применять более сложные критерии, например, соотношение суммы займа и ежемесячного дохода.